# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk import word_tokenize
from nltk import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import pickle 
import re 
import nltk
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

In [5]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
print(stopwords.words('english'))
 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# load data from database
engine = create_engine('sqlite:///disaster_project.db')

sql = 'select * from cleaned_messages'
#table is cleaned_messages
df = pd.read_sql_query(sql,engine)
y = df.drop('message',axis=1) 
y = y.drop('id',axis=1) 
y = y.drop('original',axis=1) 
y = y.drop('genre',axis=1)
y = y.astype('int')
X = df['message'] 

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#add to X, make dummies. could this improve the model? 
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9\s]',' ',text)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = []
    for tok in tokens:
        clean_tok = WordNetLemmatizer().lemmatize(tok,).lower().strip()
        cleaned_tokens.append(clean_tok)
        
    return cleaned_tokens
        
    

In [9]:
toktest = tokenize(df['message'].iloc[0])
toktest

['weather',
 'update',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

In [10]:
nas = df.isna().sum()
nas[nas>0]

original    16046
dtype: int64

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize,token_pattern=None)),
                    ('Tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
#this takes some time to run. Over 30 minutes. 
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x000001A0F016DC10>)),
                ('Tfidf', TfidfTransformer()),
                ('rf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [14]:
# with open('interim_model.pkl','wb') as f:
#     pickle.dump(pipeline,f)

In [15]:
import joblib
joblib.dump(pipeline, 'interim_model3.pkl')

['interim_model3.pkl']

In [16]:
y_test.shape

(5244, 36)

In [17]:
#TODO change genre column.... 
y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
15305,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16171,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
13263,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
18542,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [18]:
y_pred = pipeline.predict(X_test)
y_pred[:5]
#y_pred = pd.DataFrame(y_pred)

array([[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [19]:
y_pred.shape

(5244, 36)

In [20]:
ycols = y_test.columns.tolist()
y_pred2 = pd.DataFrame(y_pred,columns=ycols)
#y_test[ycols[i]]
type(y_pred2)

pandas.core.frame.DataFrame

In [21]:
y_pred2.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
print(y_test.columns)
print(y_pred2.columns)
print(set(y_test.columns)-set(y_pred2.columns))

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')
Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_r

In [23]:
#TODO need to add the F1 score too. 
#TODO convert this to a definition 
#note: need a division by zero catch. 
l = []
for i in range(len(ycols)):
    
    accuracy=accuracy_score(y_test[ycols[i]],y_pred2[ycols[i]])
    precision=precision_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted',zero_division=1)
    recall=recall_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted')
    f1score = f1_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted')    
    temp ={'Column':[ycols[i]]
                     ,'Accuracy':[accuracy]
                     ,'Precision':[precision],
                     'Recall':[recall],
                      'F1 Score':[f1score]}
    
    temp = pd.DataFrame(temp)
    #temp = {k:[v] for k,v in temp.items()}  
    l.append(temp)
print(len(l))
dfmetrics=pd.concat(l,axis=0)
dfmetrics


36


,Column,Accuracy,Precision,Recall,F1 Score
0,related,0.807208,0.802477,0.807208,0.771500
0,request,0.896834,0.894631,0.896834,0.882287
0,offer,0.995995,0.996011,0.995995,0.993997
0,aid_related,0.783562,0.783410,0.783562,0.778883
0,medical_help,0.926773,0.913851,0.926773,0.897372
0,medical_products,0.953852,0.943984,0.953852,0.933928
0,search_and_rescue,0.974256,0.974920,0.974256,0.962273
0,security,0.984554,0.977251,0.984554,0.977261
0,military,0.971587,0.969576,0.971587,0.959284
0,child_alone,1.000000,1.000000,1.000000,1.000000


In [24]:
#looking at average accuracy, precision, recalls since over 30 output variables. 
dfmetrics.describe()

,Accuracy,Precision,Recall,F1 Score
count,36.000000,36.000000,36.000000,36.000000
mean,0.948290,0.943340,0.948290,0.935090
std,0.051425,0.054081,0.051425,0.057969
min,0.783562,0.783410,0.783562,0.771500
25%,0.936308,0.927296,0.936308,0.922440
50%,0.957952,0.957489,0.957952,0.945519
75%,0.984649,0.981379,0.984649,0.977565
max,1.000000,1.000000,1.000000,1.000000


In [25]:
# penmodel = None
# with open("penultimate_classifier.pkl", "wb") as f:
#     pickle.dump(penmodel, f)

### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
''' For reference. 
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize,token_pattern=None)),
                    ('Tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(RandomForestClassifier()))])
'''
#NOTE having alot of combinations takes a LONG TIME. with over 20 combos, it was still running after leaving it on overnight. 

parameters = {'vect__min_df':[1,2] 
,       'vect__max_df':[0.99,0.95]} 
#,                'rf__max_depth':[1,2,4,8],
#             'rf__max_leaf_nodes':[None,2,4,8]}

cv = GridSearchCV(pipeline,param_grid=parameters)
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x000001A0F016DC10>)),
                                       ('Tfidf', TfidfTransformer()),
                                       ('rf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'vect__max_df': [0.99, 0.95], 'vect__min_df': [1, 2]})

In [ ]:
# usegridsearch = GridSearchCV(estimator=RandomForestClassifier(), 
#                     param_grid=parameters,
#                     cv=5,
#                     refit=True,
#                     error_score=0,
#                     n_jobs=-1)

cv.fit(X_train, y_train)
y_pred_improve = cv.predict(X_test)
y_pred_improve[:5]

In [ ]:
#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
#Ccvfc.fit(X, y)
print (cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
#TODO make a division by zero catch. Done 

l = []
for i in range(len(ycols)):
    
    accuracy=accuracy_score(y_test[ycols[i]],y_pred_improve[ycols[i]])
    precision=precision_score(y_test[ycols[i]],y_pred_improve[ycols[i]],average='weighted',zero_division=1)
    recall=recall_score(y_test[ycols[i]],y_pred_improve[ycols[i]],average='weighted')
    temp ={'Column':[ycols[i]]
                     ,'Accuracy':[accuracy]
                     ,'Precision':[precision],
                     'Recall':[recall]}
    
    temp = pd.DataFrame(temp)
    #temp = {k:[v] for k,v in temp.items()}  
    l.append(temp)
print(len(l))
dfmetrics=pd.concat(l,axis=0)
dfmetrics

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline2 = Pipeline([('vect',CountVectorizer(tokenizer=tokenize,token_pattern=None)),
                    ('Tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(LogisticRegression()))])
pipeline3 = Pipeline([('vect',CountVectorizer(tokenizer=tokenize,token_pattern=None)),
                    ('Tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(KNeighborsClassifier()))])

### 9. Export your model as a pickle file

In [ ]:

finalmodel = None
with open("emergency_type_classifier.pkl", "wb") as f:
    pickle.dump(finalmodel, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.